In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
s1 = pd.read_csv('s1_data.csv')
s2 = pd.read_csv('s2_data.csv')

In [3]:
# sort s1 and s2 by date 
s1 = s1.sort_values(by=['date'])
s2 = s2.sort_values(by=['date'])

In [4]:
s1.head()

,geometry,label,date,tile_number,id,location_id,image_dir
131,"[[[30.033794, -1.947753], [30.033877, -1.99400...",0,2018-12-15,25,sen12floods_s1_labels_0025_2018_12_15,25,sen12flood/sen12floods_s1_source/sen12floods_s...
121,"[[[30.125653, -1.901331], [30.125737, -1.94758...",0,2018-12-15,24,sen12floods_s1_labels_0024_2018_12_15,24,sen12flood/sen12floods_s1_source/sen12floods_s...
141,"[[[30.079766, -1.947669], [30.079851, -1.99392...",0,2018-12-15,26,sen12floods_s1_labels_0026_2018_12_15,26,sen12flood/sen12floods_s1_source/sen12floods_s...
101,"[[[30.033712, -1.901496], [30.033794, -1.94775...",0,2018-12-15,22,sen12floods_s1_labels_0022_2018_12_15,22,sen12flood/sen12floods_s1_source/sen12floods_s...
111,"[[[30.079683, -1.901415], [30.079766, -1.94766...",0,2018-12-15,23,sen12floods_s1_labels_0023_2018_12_15,23,sen12flood/sen12floods_s1_source/sen12floods_s...


In [5]:
!mkdir TimewiseCSV

In [7]:
# for same date create csv files
base_path= "TimewiseCSV\\"
k={}
for i in range(len(s1)):
    date = s1['date'][i]
    k[date]=0
    k[date]+=1
    s1_date = s1[s1['date']==date]
    s2_date = s2[s2['date']==date]
    if(k[date]==1):
        s1_date.to_csv(base_path+str(date)+'_s1.csv',index=False)
        s2_date.to_csv(base_path+str(date)+'_s2.csv',index=False)
